In [1]:
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from JSON_converter import extraction_json

In [2]:
#Loading the model
nlp=spacy.load('en_core_web_sm')


#Getting the existing pipeline component
ner=nlp.get_pipe("ner")

In [3]:
# training data
#TO REPLACE WITH THE JSON PROCESSING FUNCTION
TRAIN_DATA = extraction_json()

('COLLEGE STATION Texas A&M Forest Service issued a press release that it readies firefighting resources in anticipation of an increase in wildfire activity caused by pre-frontal conditions approaching the state. The fire environment will include elevated to critical fire weather, with above normal temperatures and wind speeds near 20 mph, aligned with freeze-cured grasses across the landscape, the press release stated. This combination will support increased wildfire activity. Forecast fire danger will be high to very high for portions of the Rolling Plains and Cross Timbers regions, specifically for areas around Plainview, Wichita Falls, Weatherford, Lampasas and San Angelo. For current conditions and wildfire outlook, visit the Texas Fire Potential Outlook at https://bit.ly/3kemhbG. State and local firefighters, including Texas A&M Forest Service and Texas Intrastate Fire Mutual Aid System, responded to 97 wildfires since Friday morning for an estimated 7,460 acres burned. Strong no

NameError: name 'TRAIN_DATA' is not defined

In [ ]:
# Adding labels to the NER

for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

TypeError: 'NoneType' object is not iterable

In [ ]:
# Disable pipeline components that we don't need to modify
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
            texts, annotations = zip(*batch)
            
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            
            # Update the model
            nlp.update(example, drop=0.5, losses=losses)

C:\Users\SOR Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I got my truck fixed" with entities "[(16, 21, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\SOR Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "I reached Chennai yesterday." with entities "[(19, 28, 'GPE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
C:\Users\SOR Lucas\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the tex

In [ ]:
# Testing the model
doc = nlp("Despite snow on the ground and freezing temperatures, Alberta's wildfire season officially started on Tuesday and the government is preparing for the season by hiring hundreds of staff including firefighters, weather specialists and fire behaviour analysts. Alberta Wildfire says the percentage of man-made fires in the province has dropped, even with the increase in outdoor activity the past two summers due to COVID-19. In 2020, 88 per cent of fires were caused by humans. That dropped to 62 per cent in 2021. 'We need to maintain that vigilance and improve upon the gains that we made last year so we can minimize the risk of wildfire,' said wildfire information officer Derrick Forsythe. Alberta Wildfire Information Officer, Derrick Forsythe, encourages people to remain vigilant when using campfires or off highway vehicles. (CBC/Google)")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Alto', 'PRODUCT')]


In [ ]:
# Save the  model to directory
output_dir = './NER_model/'
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to ./NER_models/


In [ ]:
#Loading the model
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Fridge can be ordered in FlipKart" )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Loading from ./NER_models/
Entities [('Fridge', 'PRODUCT'), ('FlipKart', 'ORG')]
